# The TLS214_NPL_PUBLN Table 

This notebook explores the ``TLS214_NPL_PUBLN`` table, which contains crucial information on **non-patent literature (NPL)** citations associated with patents. Non-patent literature includes references to scientific articles, technical standards, and online documents that are not classified as patents but significantly contribute to the background knowledge of an invention. ``TLS214_NPL_PUBLN`` provides unstructured bibliographic information about these citations, categorizing and, where possible, structuring details like authors, titles, publishers, dates, and identifiers such as DOIs, ISBNs, and ISSNs.

The ``NPL_PUBLN_ID`` is the primary identifier for each NPL citation. Additional fields, including ``NPL_TYPE``, ``NPL_AUTHOR``, ``NPL_TITLE1``, and ``NPL_PUBLN_DATE``, help capture detailed bibliographic data depending on the NPL type. Types are assigned according to citation categories such as book citations, journal citations, and web-based citations, which dictate which fields are likely to be populated.

Each type of NPL citation (e.g., journal articles, online sources, and technical reports) comes with a varying degree of structured metadata. For example, journal citations often include detailed fields like ``NPL_AUTHOR``, ``NPL_VOLUME``, ``NPL_ISSUE``, and ``NPL_PAGE_FIRST``, which provide a rich basis for analysis. Online citations may contain attributes specific to web sources, such as ``ONLINE_AVAILABILITY`` and ``ONLINE_SEARCH_DATE``, reflecting the rise in digital and database citations in recent years.

Analysing the ``TLS214_NPL_PUBLN`` table allows for a deeper understanding of the scientific and technical background influencing patent documents. Non-patent literature often indicates the foundation of emerging technologies, and cross-referencing these citations with patent publications can reveal interdisciplinary trends and areas where patented inventions are built upon or diverge from existing research.








In [2]:
from epo.tipdata.patstat import PatstatClient
from epo.tipdata.patstat.database.models import (
    TLS201_APPLN,
    TLS212_CITATION,
    TLS211_PAT_PUBLN,
    TLS214_NPL_PUBLN
)
from sqlalchemy import and_, case, func, select

# Initialise the PATSTAT client
patstat = PatstatClient(env="TEST")

# Access ORM
db = patstat.orm()

## Key Fields in the TLS214_NPL_PUBLN Table

### NPL_PUBLN_ID 
The attribute ``NPL_PUBLN_ID``is the primary key to uniquely identify non-patent literature (NPL) citations. 

### XP_NR

The ``XP_NR`` attribute, also known as the XP number, uniquely identifies non-patent literature (NPL) items that have been used by EPO examiners. These items—like research articles, books, or webpages—are given a specific accession number by the EPO for internal tracking. This number is recorded in the XP_NR field in DOCDB and PATSTAT databases and is used to distinguish NPL citations that have played a role in prior art searches within patent examination procedures.

The ``XP_NR`` helps track NPLs cited by EPO examiners, giving each a unique identifier unless they have only been used by other offices. If an NPL has not been referenced by the EPO, it is assigned the default value of 0.

This number is also valuable in cases where an NPL refers to a patent document, such as when an NPL citation includes a reference to a specific patent publication, which can be cross-referenced with the ``CITED_PAT_PUBLN_ID`` in ``TLS212_CITATION``.


### NPL_TYPE
The ``NPL_TYPE`` attribute in the PATSTAT database categorizes types of non-patent literature (NPL) citations. It distinguishes between various formats and sources of NPL data based on its nature and structure:

1. General Abstracts:

    - **a (Abstract citation of no specific kind)**: This type indicates NPL entries with minimal information, often lacking a structured or "rich" format. These citations may refer to broad or generalized abstracts without specifying the exact source type.

2. Printed Literature:

    - **b (Book citation)**: Used for references to books.
    - **c (Chemical abstracts citation)**: Refers specifically to abstracts from chemical literature.
    - **i (Biological abstracts citation)**: Denotes citations from biological abstracts.
    - **j (Patent Abstracts of Japan citation)**: Indicates that the citation comes from the "Patent Abstracts of Japan" database.
    - **s (Serial / Journal / Periodical citation)**: Used for citations from academic or professional journals, serials, or periodicals.

3. Online and Database Sources:

    - **d (Derwent citation)**: Citations sourced from the Derwent database.
    - **e (Database citation)**: Used for citations from various databases, distinguishing them from web-based sources.
    - **w (World Wide Web / Internet search citation)**: Refers to NPL entries sourced from internet searches or online publications.

4. Dummy Entry:

    - **space**: This empty designation is used as a dummy placeholder, typically where a citation lacks a specific type.
   
The default value for ``NPL_TYPE`` is "**a**", assigned to general abstracts with limited structural data. This attribute helps classify NPL citations by source type, which supports more granular analysis of cited literature across patents.

### NPL_BIBLIO

This attribute contains bibliographic data related to Non-Patent Literature (NPL). It serves as a record of citations that reference NPL materials, which may include books, articles, reports, and other scholarly works not covered by patents.


In [2]:
sample_npl_records = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

sample_npl_res = patstat.df(sample_npl_records)
sample_npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio
0,0000269DA4B7AFB6D0716DDAF91F973D,None,a,"Kang, Tao et al., 'Determining Natural Arm Con..."
1,00004E194997474EA454E861B2734E01,None,a,"Huffman, John Pearley, 'GEM E825', Aug. 2002, ..."
2,0000D32AE73D53FA12E9D8A639D6F090,None,a,“Injecting Trust to Protect Revenue and Reputa...
3,0000F34A4E9A83AD5FE491609D1FE546,None,a,PSI Sinewave Interactive Inverter Charger Tech...
4,0000F465A69E7B343B5B32C5B688E334,None,s,"徐洪涛等: '利用尖劈和粗糙元技术模拟大气边界层的研究', 《公路交通科技》, vol. 2..."
...,...,...,...,...
291040,FFFEA59B2225461BB0101D6A7F5D7807,None,a,Chinese Office Action for Application No. 2009...
291041,FFFEFDFF2E80CE58EE8DA22093DCBBD1,None,a,International Search Report of PCT/JP2021/0385...
291042,FFFF1FE7D846113F48669D4E18995558,None,a,"European Search Reports dated Jan. 5, 2017 cor..."
291043,FFFF64649588E1947EB8585409D122E0,None,a,"Froehlich, Jon , et al. , “Disaggregated End-U..."


There is no xp_nr they are all NONE

In [3]:
query_npl_with_xp_nr = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio
    )
    .filter(TLS214_NPL_PUBLN.xp_nr != None)  # Filter for entries where XP_NR is not zero
)

# Convert the result to a DataFrame
npl_with_xp_nr_res = patstat.df(query_npl_with_xp_nr)

# Display the resulting DataFrame
npl_with_xp_nr_res

""


In [4]:
npl_citation_count = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio,
        func.count(TLS212_CITATION.cited_npl_publn_id).label('citation_count')  # Count citations
    )
    .join(TLS212_CITATION, TLS214_NPL_PUBLN.npl_publn_id == TLS212_CITATION.cited_npl_publn_id)  # Join condition
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')  # Exclude NPL publn_id equal to 0
    .group_by(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio
    )
    .order_by(func.count(TLS212_CITATION.cited_npl_publn_id).label('citation_count').desc())  # Order by NPL publn_id
)

# Convert the result to a DataFrame
npl_citation_count_res = patstat.df(npl_citation_count)

# Display the resulting DataFrame
npl_citation_count_res


,npl_publn_id,xp_nr,npl_type,npl_biblio,citation_count
0,AC20122300125A92827A632E26F55A2F,None,a,"Sommerfeld, A. , “On the propagation of electr...",275
1,1D54EED20D327BDB9E8AD38FB1A3DA83,None,a,"Elmore, Glenn , “Introduction to the Propagati...",258
2,99BE5D6450F208B4D64B9DA0EA63AB50,None,a,"Ren-Bin, Zhong et al., “Surface plasmon wave p...",252
3,BDE01A9CB0674AE9E2C6DA37E5002EF1,None,a,"Barlow, H. M. et al., “Surface Waves”, 621.396...",250
4,B0E309B6129FD644E5F67B8B4D305ECC,None,a,"Friedman, M. et al., “Low-Loss RF Transport Ov...",241
...,...,...,...,...,...
291040,A69CC8F2281BED068171299C488E568B,None,w,BASU B J ET AL: 'A novel pyrene-based binary p...,1
291041,D0A1143EBB7DE62128CB6726C52BD1DD,None,w,FORSSELL U.: 'Closed-loop Identification Metho...,1
291042,27811AFB287B02BB9AFF33C6873A605E,None,w,"STACK ET AL., CONCEPTUAL DESIGN AND PERFORMANC...",1
291043,A33FA71CED86CF302A123561A9839963,None,w,GRAVES ALEX ET AL: 'Speech recognition with de...,1


In [5]:
# Specify the NPL publication ID you want to check
specific_npl_publn_id = 'AC20122300125A92827A632E26F55A2F'

# Query to join NPL and Citation tables for the specific NPL
detailed_npl_citations = (
    db.query(
        TLS212_CITATION.pat_publn_id,                # Patent publication ID citing the NPL
        TLS212_CITATION.cited_npl_publn_id,         # The cited NPL publication ID
        TLS212_CITATION.citn_id,                     # Citation ID                 # Citation sequence number
        TLS212_CITATION.citn_origin,
        TLS212_CITATION.npl_citn_seq_nr,            
        TLS214_NPL_PUBLN.xp_nr,                      # Unique identifier for the NPL
        TLS214_NPL_PUBLN.npl_type,                   # Type of the NPL
        TLS214_NPL_PUBLN.npl_biblio                  # Bibliographic data of the NPL
    )
    .join(TLS214_NPL_PUBLN, TLS212_CITATION.cited_npl_publn_id == TLS214_NPL_PUBLN.npl_publn_id)  # Join condition
    .filter(TLS214_NPL_PUBLN.npl_publn_id == specific_npl_publn_id)  # Filter for the specific NPL publn_id
    .order_by(TLS212_CITATION.pat_publn_id)  # Order by patent publication ID
)

# Convert the result to a DataFrame
detailed_npl_citations_res = patstat.df(detailed_npl_citations)

# Display the resulting DataFrame
detailed_npl_citations_res


,pat_publn_id,cited_npl_publn_id,citn_id,citn_origin,npl_citn_seq_nr,xp_nr,npl_type,npl_biblio
0,423904579,AC20122300125A92827A632E26F55A2F,101,APP,12,None,a,"Sommerfeld, A. , “On the propagation of electr..."
1,440586846,AC20122300125A92827A632E26F55A2F,131,APP,29,None,a,"Sommerfeld, A. , “On the propagation of electr..."
2,445086230,AC20122300125A92827A632E26F55A2F,141,APP,37,None,a,"Sommerfeld, A. , “On the propagation of electr..."
3,451158451,AC20122300125A92827A632E26F55A2F,124,APP,20,None,a,"Sommerfeld, A. , “On the propagation of electr..."
4,471650228,AC20122300125A92827A632E26F55A2F,132,APP,30,None,a,"Sommerfeld, A. , “On the propagation of electr..."
...,...,...,...,...,...,...,...,...
270,562508515,AC20122300125A92827A632E26F55A2F,410,APP,16,None,a,"Sommerfeld, A. , “On the propagation of electr..."
271,568969669,AC20122300125A92827A632E26F55A2F,405,APP,16,None,a,"Sommerfeld, A. , “On the propagation of electr..."
272,574165585,AC20122300125A92827A632E26F55A2F,412,APP,16,None,a,"Sommerfeld, A. , “On the propagation of electr..."
273,576172092,AC20122300125A92827A632E26F55A2F,187,APP,16,None,a,"Sommerfeld, A. , “On the propagation of electr..."


In [6]:

# Check for duplicates in 'pat_publn_id'
duplicate_pat_publn_ids = detailed_npl_citations_res[detailed_npl_citations_res.duplicated(['pat_publn_id'], keep=False)]

# Display the duplicates
duplicate_pat_publn_ids


,pat_publn_id,cited_npl_publn_id,citn_id,citn_origin,npl_citn_seq_nr,xp_nr,npl_type,npl_biblio


In [7]:
query_npl_citations = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio,
        TLS212_CITATION.pat_publn_id,
        TLS212_CITATION.cited_npl_publn_id
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .join(TLS212_CITATION, TLS214_NPL_PUBLN.npl_publn_id == TLS212_CITATION.cited_npl_publn_id)  
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

# Convert the result to a DataFrame
npl_citations_res = patstat.df(query_npl_citations)

# Display the resulting DataFrame
npl_citations_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,pat_publn_id,cited_npl_publn_id
0,0000269DA4B7AFB6D0716DDAF91F973D,None,a,"Kang, Tao et al., 'Determining Natural Arm Con...",421154224,0000269DA4B7AFB6D0716DDAF91F973D
1,00004E194997474EA454E861B2734E01,None,a,"Huffman, John Pearley, 'GEM E825', Aug. 2002, ...",450469115,00004E194997474EA454E861B2734E01
2,0000D32AE73D53FA12E9D8A639D6F090,None,a,“Injecting Trust to Protect Revenue and Reputa...,447057566,0000D32AE73D53FA12E9D8A639D6F090
3,0000D32AE73D53FA12E9D8A639D6F090,None,a,“Injecting Trust to Protect Revenue and Reputa...,413978388,0000D32AE73D53FA12E9D8A639D6F090
4,0000D32AE73D53FA12E9D8A639D6F090,None,a,“Injecting Trust to Protect Revenue and Reputa...,443474748,0000D32AE73D53FA12E9D8A639D6F090
...,...,...,...,...,...,...
733798,FFFF64649588E1947EB8585409D122E0,None,a,"Froehlich, Jon , et al. , “Disaggregated End-U...",602085105,FFFF64649588E1947EB8585409D122E0
733799,FFFF64649588E1947EB8585409D122E0,None,a,"Froehlich, Jon , et al. , “Disaggregated End-U...",602085079,FFFF64649588E1947EB8585409D122E0
733800,FFFF98CFFFC1F2156FBD5022BC073C27,None,b,J.P.HIRTH; G.M. POUND: 'Condensation and evapo...,413220204,FFFF98CFFFC1F2156FBD5022BC073C27
733801,FFFF98CFFFC1F2156FBD5022BC073C27,None,b,J.P.HIRTH; G.M. POUND: 'Condensation and evapo...,413220204,FFFF98CFFFC1F2156FBD5022BC073C27


In [8]:
# Check for duplicates in 'pat_publn_id'
duplicate= npl_citations_res[npl_citations_res.duplicated(['pat_publn_id'], keep=False)]

# Display the duplicates
duplicate

,npl_publn_id,xp_nr,npl_type,npl_biblio,pat_publn_id,cited_npl_publn_id
0,0000269DA4B7AFB6D0716DDAF91F973D,None,a,"Kang, Tao et al., 'Determining Natural Arm Con...",421154224,0000269DA4B7AFB6D0716DDAF91F973D
1,00004E194997474EA454E861B2734E01,None,a,"Huffman, John Pearley, 'GEM E825', Aug. 2002, ...",450469115,00004E194997474EA454E861B2734E01
2,0000D32AE73D53FA12E9D8A639D6F090,None,a,“Injecting Trust to Protect Revenue and Reputa...,447057566,0000D32AE73D53FA12E9D8A639D6F090
3,0000D32AE73D53FA12E9D8A639D6F090,None,a,“Injecting Trust to Protect Revenue and Reputa...,413978388,0000D32AE73D53FA12E9D8A639D6F090
4,0000D32AE73D53FA12E9D8A639D6F090,None,a,“Injecting Trust to Protect Revenue and Reputa...,443474748,0000D32AE73D53FA12E9D8A639D6F090
...,...,...,...,...,...,...
733798,FFFF64649588E1947EB8585409D122E0,None,a,"Froehlich, Jon , et al. , “Disaggregated End-U...",602085105,FFFF64649588E1947EB8585409D122E0
733799,FFFF64649588E1947EB8585409D122E0,None,a,"Froehlich, Jon , et al. , “Disaggregated End-U...",602085079,FFFF64649588E1947EB8585409D122E0
733800,FFFF98CFFFC1F2156FBD5022BC073C27,None,b,J.P.HIRTH; G.M. POUND: 'Condensation and evapo...,413220204,FFFF98CFFFC1F2156FBD5022BC073C27
733801,FFFF98CFFFC1F2156FBD5022BC073C27,None,b,J.P.HIRTH; G.M. POUND: 'Condensation and evapo...,413220204,FFFF98CFFFC1F2156FBD5022BC073C27


In [9]:
query_book_citations = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio,
        func.count(TLS212_CITATION.cited_npl_publn_id).label('citation_count')
    )
    .join(TLS212_CITATION, TLS214_NPL_PUBLN.npl_publn_id == TLS212_CITATION.cited_npl_publn_id, isouter=True)
    .filter(TLS214_NPL_PUBLN.npl_type == 'b')
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .group_by(TLS214_NPL_PUBLN.npl_publn_id, TLS214_NPL_PUBLN.xp_nr, TLS214_NPL_PUBLN.npl_type, TLS214_NPL_PUBLN.npl_biblio)
    .order_by(func.count(TLS212_CITATION.cited_npl_publn_id).label('citation_count').desc()).limit(10)
)

# Convert the result to a DataFrame
book_citations_res = patstat.df(query_book_citations)

# Display the resulting DataFrame
book_citations_res



,npl_publn_id,xp_nr,npl_type,npl_biblio,citation_count
0,82AC61410D0374C35AB4CE739C3563C7,None,b,"17 March 2017, PACKT PUBLISHING , ISBN: 978-1-...",49
1,6628C2C74B9B5E60AC56834306A7A478,None,b,YIJUN LIU: 'Handbook of Polyurethane Raw Mater...,8
2,A17A81CF38BE458B60EBCD35FF5A0A6A,None,b,"'CIRP Annals - Manufacturing Technology', vol....",8
3,7424DD441B42D40EE7F090AFACF15FFE,None,b,"李春: '《现代大型风力机设计原理》', 31 January 2013, 上海科学技术出版社",7
4,676E92C1D0289370683DEC84642B6B2B,None,b,"HOUBEN WEYL: 'Makromolekulare Stoffe', vol. XI...",7
5,F4BC4BA3F20FC6F52C130AF830ADA6B0,None,b,"HAU ERICH ED - HAU E: 'Windkraftanlagen, Ausge...",6
6,4BEF4F4FB94DFF7BC5EED89B0D082C53,None,b,"PARNAS ET AL.: 'Liquid Composite Moulding', 20...",6
7,CEE9ECE3E0C59A25986DB65FED8D5676,None,b,"'Reliability Theory and Practice', 31 December...",6
8,A3C767F3A83B4D663B9C9BA7058E6EB3,None,b,"'Agnew, Chem. Intl. Ed. Engl.', vol. 33, 1994,...",6
9,25C7CA921A732BA400AA8A7D077CF6F7,None,b,"'Wind Energy Handbook', JOHN WILEY & SONS, pag...",6


In [10]:
query_web_citations = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio,
        func.count(TLS212_CITATION.cited_npl_publn_id).label('citation_count')
    )
    .join(TLS212_CITATION, TLS214_NPL_PUBLN.npl_publn_id == TLS212_CITATION.cited_npl_publn_id, isouter=True)
    .filter(TLS214_NPL_PUBLN.npl_type == 'w')
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .group_by(TLS214_NPL_PUBLN.npl_publn_id, TLS214_NPL_PUBLN.xp_nr, TLS214_NPL_PUBLN.npl_type, TLS214_NPL_PUBLN.npl_biblio)
    .order_by(func.count(TLS212_CITATION.cited_npl_publn_id).label('citation_count').desc()).limit(10)
)

# Convert the result to a DataFrame
web_citations_res = patstat.df(query_web_citations)

# Display the resulting DataFrame
web_citations_res


,npl_publn_id,xp_nr,npl_type,npl_biblio,citation_count
0,13BC922E726BC8D08D7CE25D71A55AA4,None,w,YUQING TANG: 'High Power Inverter EMI characte...,12
1,B17BD4D26471A633C674534CD23BBF6D,None,w,BACCINO FRANCESCO ET AL: 'An Optimal Model-Bas...,11
2,22A0620BB047CDEC36781DA100ED7712,None,w,MANFRED BEYER ET AL: 'New BAUER Flydrill syste...,10
3,575268A42E7AFA839B8FC7573F190158,None,w,BOSSANYI E A: 'The Design of Closed Loop Contr...,9
4,D12FA067EBD80F3BC02AADB0A18FFED0,None,w,PRILLWITZ F ET AL: 'Primärregelung mit Windkra...,8
5,A18BE4607DE089C6D036425350BA7134,None,w,LI WEN ET AL: 'Automatic Bird Species Detectio...,7
6,B2D297847DDCDAAD1E18B64561B19816,None,w,BERT DEN OUDEN: 'Electrification in the Dutch ...,7
7,B804F4EF30D63DC6954D1BFFBAAA1C63,None,w,GINGER GARDINER: 'Sensors for monitoring resin...,7
8,E87BD20B35F7CD8DF2156F4A17F93603,None,w,PRILLWITZ F ET AL: 'Primärregelung mit Windkra...,6
9,EE244465F686D96199D48F2547C93E24,None,w,SIEMENSGAMESA: 'How a small robot innovates ro...,6


### NPL_AUTHOR: 
The author of the online article or document. This attribute may only be populated for these NPL types:
- Book citation (b)
- Chemical Abstract citation (c)
- Biological abstract citation (i)
- Serial / Journal / Periodical citation (s)
- Derwent citation (d)
- Database citation (e)
- World Wide Web / Internet search citation (w)

In [4]:
npl_author = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_author != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

author_res = patstat.df(npl_author)
author_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author
0,0000F465A69E7B343B5B32C5B688E334,None,s,"徐洪涛等: '利用尖劈和粗糙元技术模拟大气边界层的研究', 《公路交通科技》, vol. 2...",徐洪涛等
1,0003477C90FCF66328A5D6030EA1F6AF,None,s,"支欣 等: '“综合能源系统的运行优化配置分析”', 《自动化与仪器仪表》, no. 08,...",支欣 等
2,0003BCB82D667DABC8B4C0C7CE188095,None,w,BORUP M ; ET AL: 'General rights Copyright and...,Borup M ; ET AL
3,000412DBB9ABCB3BCABD76B5433C6A21,None,s,"鲁朝静: '齿轮箱复合故障诊断方法的研究', 《中国优秀硕士学位论文全文数据库 工程科技Ⅱ辑...",鲁朝静
4,0005A8D2B2F68948B0296E7999FAC02C,None,s,"李卫东: '大功率缺失下主动频率响应控制初探', 《电力系统自动化》",李卫东
...,...,...,...,...,...
59645,FFFB9C35DEE47FB3DC7AAB8086799731,None,s,"张光亚等: '考虑供需互动和分布式电源运行特性的主动配电网网架规划', 《电网规划》",张光亚等
59646,FFFBD1B477D2918F38526390F9C108B1,None,w,"ANONYMOUS: 'Standard insulated heating tape', ...",ANONYMOUS
59647,FFFCB44347687FA2DADBF5BB375B563C,None,s,"徐迅等: '考虑环境成本和时序特性的微网多类型分布式电源选址定容规划', 电网技术, vol...",徐迅等
59648,FFFE2919480E05F7D3E0F62CBEB9243A,None,s,HIEU T. NGUYEN等: 'Occlusion robust adaptive te...,Hieu T. Nguyen等


### NPL_TITLE1 and NPL_TITLE2: 
Primary and secondary titles of the online article. This attribute may only be populated for these NPL types:
- Book citation (b)
- Chemical Abstract citation (c)
- Biological abstract citation (i)
- Patent Abstracts of Japan (j) (if TITLE2)
- Serial / Journal / Periodical citation (s)
- Derwent citation (d)
- Database citation (e) (if TITLE1)
- World Wide Web / Internet search citation (w)

In [7]:
npl_title = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_title1 != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_title)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2
0,0000F465A69E7B343B5B32C5B688E334,None,s,"徐洪涛等: '利用尖劈和粗糙元技术模拟大气边界层的研究', 《公路交通科技》, vol. 2...",徐洪涛等,利用尖劈和粗糙元技术模拟大气边界层的研究,《公路交通科技》
1,0003477C90FCF66328A5D6030EA1F6AF,None,s,"支欣 等: '“综合能源系统的运行优化配置分析”', 《自动化与仪器仪表》, no. 08,...",支欣 等,“综合能源系统的运行优化配置分析”,《自动化与仪器仪表》
2,0003BCB82D667DABC8B4C0C7CE188095,None,w,BORUP M ; ET AL: 'General rights Copyright and...,Borup M ; ET AL,General rights Copyright and moral rights for ...,None
3,000412DBB9ABCB3BCABD76B5433C6A21,None,s,"鲁朝静: '齿轮箱复合故障诊断方法的研究', 《中国优秀硕士学位论文全文数据库 工程科技Ⅱ辑...",鲁朝静,齿轮箱复合故障诊断方法的研究,《中国优秀硕士学位论文全文数据库 工程科技Ⅱ辑》
4,0005A8D2B2F68948B0296E7999FAC02C,None,s,"李卫东: '大功率缺失下主动频率响应控制初探', 《电力系统自动化》",李卫东,大功率缺失下主动频率响应控制初探,《电力系统自动化》
...,...,...,...,...,...,...,...
59912,FFFB9C35DEE47FB3DC7AAB8086799731,None,s,"张光亚等: '考虑供需互动和分布式电源运行特性的主动配电网网架规划', 《电网规划》",张光亚等,考虑供需互动和分布式电源运行特性的主动配电网网架规划,《电网规划》
59913,FFFBD1B477D2918F38526390F9C108B1,None,w,"ANONYMOUS: 'Standard insulated heating tape', ...",ANONYMOUS,Standard insulated heating tape,None
59914,FFFCB44347687FA2DADBF5BB375B563C,None,s,"徐迅等: '考虑环境成本和时序特性的微网多类型分布式电源选址定容规划', 电网技术, vol...",徐迅等,考虑环境成本和时序特性的微网多类型分布式电源选址定容规划,电网技术
59915,FFFE2919480E05F7D3E0F62CBEB9243A,None,s,HIEU T. NGUYEN等: 'Occlusion robust adaptive te...,Hieu T. Nguyen等,Occlusion robust adaptive template tracking,《Eighth IEEE International Conference on Compu...


### NPL_VOLUME: 
Volume number if it is part of a journal or series. This attribute may only be populated for these NPL types:
- Book citation (b)
- Chemical Abstract citation (c)
- Biological abstract citation (i)
- Patent Abstracts of Japan (j) 
- Serial / Journal / Periodical citation (s)
- Derwent citation (d)
- World Wide Web / Internet search citation (w)

In [8]:
npl_volume = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_volume != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_volume)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume
0,0000F465A69E7B343B5B32C5B688E334,None,s,"徐洪涛等: '利用尖劈和粗糙元技术模拟大气边界层的研究', 《公路交通科技》, vol. 2...",徐洪涛等,利用尖劈和粗糙元技术模拟大气边界层的研究,《公路交通科技》,26
1,00089E39915F8E31FC16F7556310C299,None,s,"金爱娟等: '并联Buck 变换器的均流控制技术', 《电子科技》, vol. 29, no...",金爱娟等,并联Buck 变换器的均流控制技术,《电子科技》,29
2,000AB94DFBAB5CA3AF12E72D75991338,None,s,"PORTER, R. R., BIOCHEM. J., vol. 73, pages 9 - 26","PORTER, R. R.",None,Biochem. J.,73
3,001EB87E00AEB933E632A174655B256D,None,s,K. J. STANDISH ET AL.: 'Aerodynamic Analysis o...,K. J. STANDISH et al.,Aerodynamic Analysis of Blunt Trailing Edge Ai...,"Journal of Solar Energy Engineering, ASME Inte...",125
4,0020E1F28668A5DBD05ADF4CDDC290C1,None,s,"雷歌, 邓飞, 刘权等: '水下航行器折叠翼展开机构设计与动力学仿真', 《鱼雷技术》, v...","雷歌, 邓飞, 刘权等",水下航行器折叠翼展开机构设计与动力学仿真,《鱼雷技术》,21
...,...,...,...,...,...,...,...,...
12576,FFF43E8DD93999E07AE9BE83C4C390CF,None,s,"屈子森等: '基于自适应虚拟阻抗的虚拟同步机功率解耦控制策略', 《电力系统自动化》, vo...",屈子森等,基于自适应虚拟阻抗的虚拟同步机功率解耦控制策略,《电力系统自动化》,42
12577,FFF4BDC2D985997443FC8604DC401B08,None,s,"赵泰峰等: '调度主站AGC软件负荷分配的研究', 《电网技术》, vol. 29, no....",赵泰峰等,调度主站AGC软件负荷分配的研究,《电网技术》,29
12578,FFF8719579CEE6643E47DE7D0A80D46E,None,s,Y. SUK. ZHANGQ. GONG: 'Theoretical and experim...,Y. SUK. ZHANGQ. GONG,Theoretical and experimental study of an elect...,Ferroelectrics,551
12579,FFFCB44347687FA2DADBF5BB375B563C,None,s,"徐迅等: '考虑环境成本和时序特性的微网多类型分布式电源选址定容规划', 电网技术, vol...",徐迅等,考虑环境成本和时序特性的微网多类型分布式电源选址定容规划,电网技术,37


### NPL_EDITOR:
Name of the editor or the author of a book. This attribute may only be populated for these NPL types:
- Book citation (b)

In [12]:
npl_editor = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_editor != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_editor)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor
0,00012979D6CC8C5A74CC57837FB54618,None,b,中华人民共和国工业和信息化部: '《QC/T 568.1-2011 汽车机械式变速器总成 台...,None,None,《QC/T 568.1-2011 汽车机械式变速器总成 台架试验方法 第1部分：微型》,None,中华人民共和国工业和信息化部
1,000333B6997E19B8C1EECC283F403B7F,None,b,"李博洋，闫芳，单士俊，周克斌: '《机械原理简明教程》', 31 March 2014, 西...",None,None,《机械原理简明教程》,None,李博洋，闫芳，单士俊，周克斌
2,0005DBD2D037C3C8856AE29D1F6CE79D,None,b,（美）朱拉金斯基（G.SZULADZINSKI）著: '《机器与结结构动力学 例题与习题》'...,None,None,《机器与结结构动力学 例题与习题》,None,（美）朱拉金斯基（G.Szuladzinski）著
3,0005F85323D4F670F3FC3DD02A526321,None,b,"李鉴增，金立标，苗方: '《宽带网络技术》', 30 September 2015",None,None,《宽带网络技术》,None,李鉴增，金立标，苗方
4,001643D74678D1AC9A5A4AAA0313AABF,None,b,"李念奎，凌杲等编著: '《铝合金材料及其热处理技术》', 30 April 2012",None,None,《铝合金材料及其热处理技术》,None,李念奎，凌杲等编著
...,...,...,...,...,...,...,...,...,...
5200,FFE7BFDF5236338D28F6F0F454A34119,None,b,PAUL SCHERZ: 'Practical Electronics for Invent...,None,None,Practical Electronics for Inventors,None,PAUL SCHERZ
5201,FFF1E2E41760E61B92111D6A0FEF4F33,None,b,"蔡伟维: '《赛欧轿车故障检修图解》', 31 May 2004, 四川科学技术出版社",None,None,《赛欧轿车故障检修图解》,None,蔡伟维
5202,FFFBA06F8F0DFE9487CEE6006BC9B228,None,b,"黄连忠: '《辽宁省首批“十二五”普通高等教育本科省级规划教材 船舶动力装置技术管理》', ...",None,None,《辽宁省首批“十二五”普通高等教育本科省级规划教材 船舶动力装置技术管理》,None,黄连忠
5203,FFFBCDC5ABCA428286BCBFF969766D7B,None,b,"中国机械工程学会焊接学会: '《焊接手册（第三版）第三卷 焊接结构》', 31 Octobe...",None,None,《焊接手册（第三版）第三卷 焊接结构》,None,中国机械工程学会焊接学会


### NPL_ISSUE:
Specifies the issue of a Non-Patent Literature.
This attribute may only be populated for these NPL types:
- Chemical Abstract citation (c)
- Biological abstract citation (i)
- Patent Abstracts of Japan (j)
- Serial / Journal / Periodical citation (s)
- Derwent citation (d)
- World Wide Web / Internet search citation (w)

In [13]:
npl_issue = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_issue != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_issue)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue
0,0000F465A69E7B343B5B32C5B688E334,None,s,"徐洪涛等: '利用尖劈和粗糙元技术模拟大气边界层的研究', 《公路交通科技》, vol. 2...",徐洪涛等,利用尖劈和粗糙元技术模拟大气边界层的研究,《公路交通科技》,26,None,9
1,0003477C90FCF66328A5D6030EA1F6AF,None,s,"支欣 等: '“综合能源系统的运行优化配置分析”', 《自动化与仪器仪表》, no. 08,...",支欣 等,“综合能源系统的运行优化配置分析”,《自动化与仪器仪表》,None,None,08
2,000412DBB9ABCB3BCABD76B5433C6A21,None,s,"鲁朝静: '齿轮箱复合故障诊断方法的研究', 《中国优秀硕士学位论文全文数据库 工程科技Ⅱ辑...",鲁朝静,齿轮箱复合故障诊断方法的研究,《中国优秀硕士学位论文全文数据库 工程科技Ⅱ辑》,None,None,04
3,00089E39915F8E31FC16F7556310C299,None,s,"金爱娟等: '并联Buck 变换器的均流控制技术', 《电子科技》, vol. 29, no...",金爱娟等,并联Buck 变换器的均流控制技术,《电子科技》,29,None,1
4,0020E1F28668A5DBD05ADF4CDDC290C1,None,s,"雷歌, 邓飞, 刘权等: '水下航行器折叠翼展开机构设计与动力学仿真', 《鱼雷技术》, v...","雷歌, 邓飞, 刘权等",水下航行器折叠翼展开机构设计与动力学仿真,《鱼雷技术》,21,None,02
...,...,...,...,...,...,...,...,...,...,...
15819,FFE9C5C5907793A8ADB169B5FB4EB0E6,None,s,"陈中等: '基于“区域阻尼”的互联电网阻尼分析与控制的研究', 《电力系统保护与控制》",陈中等,基于“区域阻尼”的互联电网阻尼分析与控制的研究,《电力系统保护与控制》,40,None,2
15820,FFF43E8DD93999E07AE9BE83C4C390CF,None,s,"屈子森等: '基于自适应虚拟阻抗的虚拟同步机功率解耦控制策略', 《电力系统自动化》, vo...",屈子森等,基于自适应虚拟阻抗的虚拟同步机功率解耦控制策略,《电力系统自动化》,42,None,17
15821,FFF4BDC2D985997443FC8604DC401B08,None,s,"赵泰峰等: '调度主站AGC软件负荷分配的研究', 《电网技术》, vol. 29, no....",赵泰峰等,调度主站AGC软件负荷分配的研究,《电网技术》,29,None,18
15822,FFF8719579CEE6643E47DE7D0A80D46E,None,s,Y. SUK. ZHANGQ. GONG: 'Theoretical and experim...,Y. SUK. ZHANGQ. GONG,Theoretical and experimental study of an elect...,Ferroelectrics,551,None,1


### NPL_PUBLN_DATE: 
Publication date of the cited material. This attribute may only be populated for these NPL types:
- Book citation (b)
- Chemical Abstract citation (c)
- Biological abstract citation (i)
- Patent Abstracts of Japan (j) 
- Serial / Journal / Periodical citation (s)
- Derwent citation (d)
- Database citation (e) 
- World Wide Web / Internet search citation (w)

In [14]:
npl_date = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_publn_date != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_date)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date
0,0000F465A69E7B343B5B32C5B688E334,None,s,"徐洪涛等: '利用尖劈和粗糙元技术模拟大气边界层的研究', 《公路交通科技》, vol. 2...",徐洪涛等,利用尖劈和粗糙元技术模拟大气边界层的研究,《公路交通科技》,26,None,9,20090930
1,00012979D6CC8C5A74CC57837FB54618,None,b,中华人民共和国工业和信息化部: '《QC/T 568.1-2011 汽车机械式变速器总成 台...,None,None,《QC/T 568.1-2011 汽车机械式变速器总成 台架试验方法 第1部分：微型》,None,中华人民共和国工业和信息化部,None,20110518
2,00032A0C40A369642B3C6DA6663D472F,None,b,"'ASTM C172 / C172M-14a, Standard Practice for ...",None,None,"ASTM C172 / C172M-14a, Standard Practice for S...",None,None,None,2014
3,000333B6997E19B8C1EECC283F403B7F,None,b,"李博洋，闫芳，单士俊，周克斌: '《机械原理简明教程》', 31 March 2014, 西...",None,None,《机械原理简明教程》,None,李博洋，闫芳，单士俊，周克斌,None,20140331
4,0003BCB82D667DABC8B4C0C7CE188095,None,w,BORUP M ; ET AL: 'General rights Copyright and...,Borup M ; ET AL,General rights Copyright and moral rights for ...,None,None,None,None,20020101
...,...,...,...,...,...,...,...,...,...,...,...
22583,FFFBA06F8F0DFE9487CEE6006BC9B228,None,b,"黄连忠: '《辽宁省首批“十二五”普通高等教育本科省级规划教材 船舶动力装置技术管理》', ...",None,None,《辽宁省首批“十二五”普通高等教育本科省级规划教材 船舶动力装置技术管理》,None,黄连忠,None,20170228.0
22584,FFFBCDC5ABCA428286BCBFF969766D7B,None,b,"中国机械工程学会焊接学会: '《焊接手册（第三版）第三卷 焊接结构》', 31 Octobe...",None,None,《焊接手册（第三版）第三卷 焊接结构》,None,中国机械工程学会焊接学会,None,20071031.0
22585,FFFBD1B477D2918F38526390F9C108B1,None,w,"ANONYMOUS: 'Standard insulated heating tape', ...",ANONYMOUS,Standard insulated heating tape,None,None,None,None,20061101.0
22586,FFFE2919480E05F7D3E0F62CBEB9243A,None,s,HIEU T. NGUYEN等: 'Occlusion robust adaptive te...,Hieu T. Nguyen等,Occlusion robust adaptive template tracking,《Eighth IEEE International Conference on Compu...,1,None,None,20010714.0


### NPL_PUBLN_END_DATE: 
This field captures the end date of publication for the Non-Patent Literature. This attribute may only be populated for these NPL types: 
- Serial / Journal / Periodical citation (s) 
- World Wide Web / Internet search citation (w)

In [15]:
npl_enddate = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_publn_end_date != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_enddate)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date,npl_publn_end_date
0,01BDD7CAE56DDFCE28981E81EC693427,None,s,"PETOSA A., THIRAKOUNC S., ZULIANI M., ITTIPIBO...","Petosa A., Thirakounc S., Zuliani M., Ittipibo...",Comparison between planar arrays of perforated...,"ANTENNAS AND PROPAGATION SOCIETY SYMPOSIUM, 20...",2A,None,None,20050703,20050708.0
1,01CE70F9EF603E6822AB0DE0957964F4,None,s,BECHERT ET AL.: 'Drag reduction of airfoils wi...,BECHERT et al.,Drag reduction of airfoils with miniflaps. Can...,"AIAA, Fluids 2000",None,None,None,20000619,20000622.0
2,024F6648436FB726E1D2CB8FC48AEBF7,None,w,EROMON ET AL: 'Distributed Energy Resource (DE...,EROMON ET AL,"Distributed Energy Resource (DER) Using FACTS,...",2005 NATIONAL ASSOCIATION OF INDUSTRIAL TECHNO...,None,None,None,20051116,20051119.0
3,03A5FB962676A674B6BFB6625393B88D,None,s,HAN JU CHA ET AL.: 'A New Soft Switching Direc...,HAN JU CHA ET AL.,A New Soft Switching Direct Converter for Resi...,IAS 2004,2,None,None,20041003,20041007.0
4,046C4BE4ABDD0F69BFB583C3F0016284,None,s,M. LACKNER ET AL.: 'A Comparison of Smart Roto...,M. LACKNER et al.,A Comparison of Smart Rotor Control Approaches...,47th AIAA Aerospace Sciences Meeting Including...,None,None,None,20090105,20090108.0
...,...,...,...,...,...,...,...,...,...,...,...,...
204,F7D70882A7A0E2845084FA3D4C677DD4,None,s,MARTINEZ ET AL: 'Design and coordination of a ...,MARTINEZ ET AL,Design and coordination of a capacitor and on-...,WIND ENERGY,15,None,4,201205,201205
205,F828E982664B5304C3470C1BA08AA12A,None,s,COLQUHOUN ET AL: 'MMICs for Automotive and Tra...,COLQUHOUN ET AL,MMICs for Automotive and Traffic Applications,IEEE GAAS INTEGRATED CIRCUIT SYMPOSIUM - TECHN...,None,None,None,None,19921007.0
206,F82E5035CD11E0CFEF3366DCB7B7089D,None,s,"CAIRNS DOUGLAS S, BUNDY BRYAN C: 'The Applicat...","CAIRNS DOUGLAS S, BUNDY BRYAN C",The Application of Pre-Cured Carbon Fiber/Epox...,"AIAA AEROSPACE SCIENCES MEETING AND EXHIBIT, US",AIAA 2007-821,None,45th,20070108.0,20070111.0
207,FB00C0DFE65CC26EAD796FA5E2CDCA08,None,s,I. WEISS ET AL.: 'A new PV system technology-t...,I. WEISS ET AL.,A new PV system technology-the development of ...,16TH EUROPEAN PHOTOVOLTAIC SOLAR ENERGY CONFER...,III,None,None,20000501.0,20000505.0


### NPL_PUBLISHER: 
Publisher of the document. This attribute may only be populated for these NPL types:
- Book citation (b)
- Derwent citation (d)
- Database citation (e) 
- World Wide Web / Internet search citation (w)

In [16]:
npl_publisher = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_publisher != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_publisher)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date,npl_publn_end_date,npl_publisher
0,00032A0C40A369642B3C6DA6663D472F,None,b,"'ASTM C172 / C172M-14a, Standard Practice for ...",None,None,"ASTM C172 / C172M-14a, Standard Practice for S...",None,None,None,2014,None,AMERICAN SOCIETY OF THE INTERNATIONAL
1,000333B6997E19B8C1EECC283F403B7F,None,b,"李博洋，闫芳，单士俊，周克斌: '《机械原理简明教程》', 31 March 2014, 西...",None,None,《机械原理简明教程》,None,李博洋，闫芳，单士俊，周克斌,None,20140331,None,西安：西北工业大学出版社
2,0005DBD2D037C3C8856AE29D1F6CE79D,None,b,（美）朱拉金斯基（G.SZULADZINSKI）著: '《机器与结结构动力学 例题与习题》'...,None,None,《机器与结结构动力学 例题与习题》,None,（美）朱拉金斯基（G.Szuladzinski）著,None,19851231,None,北京：机械工业出版社
3,00268C6B6D77846FC9996B9CAF51924E,None,b,"顾煜炯编著: '《发电设备状态维修理论与技术》', 31 October 2009, 中国电...",None,None,《发电设备状态维修理论与技术》,None,顾煜炯编著,None,20091031,None,中国电力出版社
4,0086839FC2EC27E9854EC1D4047C01E6,None,b,"ERICH HAU: 'Grundlagen, Technik, Einsatz von W...",None,Standardwerk der Fachliteratur über Windenergi...,"Grundlagen, Technik, Einsatz von Windkraftanlagen",None,ERICH HAU,None,2008,None,SPRINGER- VERLAG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4342,FFE2B6BD03CFA050EC567D604D401D78,None,b,"凌云星 等: '《实用油墨技术指南》', 30 November 2007, 印刷工业出版社",None,None,《实用油墨技术指南》,None,凌云星 等,None,20071130,None,印刷工业出版社
4343,FFE7BFDF5236338D28F6F0F454A34119,None,b,PAUL SCHERZ: 'Practical Electronics for Invent...,None,None,Practical Electronics for Inventors,None,PAUL SCHERZ,None,2007.0,None,MCGRAW-HILL
4344,FFF1E2E41760E61B92111D6A0FEF4F33,None,b,"蔡伟维: '《赛欧轿车故障检修图解》', 31 May 2004, 四川科学技术出版社",None,None,《赛欧轿车故障检修图解》,None,蔡伟维,None,20040531.0,None,四川科学技术出版社
4345,FFFBCDC5ABCA428286BCBFF969766D7B,None,b,"中国机械工程学会焊接学会: '《焊接手册（第三版）第三卷 焊接结构》', 31 Octobe...",None,None,《焊接手册（第三版）第三卷 焊接结构》,None,中国机械工程学会焊接学会,None,20071031.0,None,机械工业出版社


### NPL_PAGE_FIRST and NPL_PAGE_LAST: 
The starting and ending pages if applicable. This attribute may only be populated for these NPL types:
- Book citation (b)
- Chemical Abstract citation (c) (if PAGE_FIRST)
- Biological abstract citation (i) (if PAGE_FIRST)
- Serial / Journal / Periodical citation (s)
- Derwent citation (d) (if PAGE_FIRST)
- World Wide Web / Internet search citation (w)

In [17]:
npl_pages = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher,
        TLS214_NPL_PUBLN.npl_page_first,
        TLS214_NPL_PUBLN.npl_page_last
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_page_first != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_pages)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date,npl_publn_end_date,npl_publisher,npl_page_first,npl_page_last
0,0000F465A69E7B343B5B32C5B688E334,None,s,"徐洪涛等: '利用尖劈和粗糙元技术模拟大气边界层的研究', 《公路交通科技》, vol. 2...",徐洪涛等,利用尖劈和粗糙元技术模拟大气边界层的研究,《公路交通科技》,26,None,9,20090930,None,None,76,79
1,00032A0C40A369642B3C6DA6663D472F,None,b,"'ASTM C172 / C172M-14a, Standard Practice for ...",None,None,"ASTM C172 / C172M-14a, Standard Practice for S...",None,None,None,2014,None,AMERICAN SOCIETY OF THE INTERNATIONAL,20,None
2,0003477C90FCF66328A5D6030EA1F6AF,None,s,"支欣 等: '“综合能源系统的运行优化配置分析”', 《自动化与仪器仪表》, no. 08,...",支欣 等,“综合能源系统的运行优化配置分析”,《自动化与仪器仪表》,None,None,08,None,None,None,132,137
3,0003BCB82D667DABC8B4C0C7CE188095,None,w,BORUP M ; ET AL: 'General rights Copyright and...,Borup M ; ET AL,General rights Copyright and moral rights for ...,None,None,None,None,20020101,None,None,1,7
4,000412DBB9ABCB3BCABD76B5433C6A21,None,s,"鲁朝静: '齿轮箱复合故障诊断方法的研究', 《中国优秀硕士学位论文全文数据库 工程科技Ⅱ辑...",鲁朝静,齿轮箱复合故障诊断方法的研究,《中国优秀硕士学位论文全文数据库 工程科技Ⅱ辑》,None,None,04,None,None,None,029,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18959,FFF19B4D2347260A9BE6121DCD8F1754,None,s,MAHFOUZ A A ET AL: 'Unity power factor operati...,MAHFOUZ A A ET AL,Unity power factor operation for 3-phase induc...,"INDUSTRIAL ELECTRONICS, CONTROL, AND INSTRUMEN...",None,None,None,19951106,None,None,300,305
18960,FFF43E8DD93999E07AE9BE83C4C390CF,None,s,"屈子森等: '基于自适应虚拟阻抗的虚拟同步机功率解耦控制策略', 《电力系统自动化》, vo...",屈子森等,基于自适应虚拟阻抗的虚拟同步机功率解耦控制策略,《电力系统自动化》,42,None,17,None,None,None,58,64
18961,FFF8719579CEE6643E47DE7D0A80D46E,None,s,Y. SUK. ZHANGQ. GONG: 'Theoretical and experim...,Y. SUK. ZHANGQ. GONG,Theoretical and experimental study of an elect...,Ferroelectrics,551,None,1,20191000.0,None,None,60,73
18962,FFFCB44347687FA2DADBF5BB375B563C,None,s,"徐迅等: '考虑环境成本和时序特性的微网多类型分布式电源选址定容规划', 电网技术, vol...",徐迅等,考虑环境成本和时序特性的微网多类型分布式电源选址定容规划,电网技术,37,None,4,None,None,None,914,921


### NPL_ABSTRACT_NR: 
This attribute stores an abstract or summary identifier related to the Non-Patent Literature (NPL) citation. This attribute may only be populated for these NPL types:
- Chemical Abstract citation (c)
- Biological abstract citation (i)
- Patent Abstracts of Japan (j) 
- Derwent citation (d)
- Database citation (e) 
- World Wide Web / Internet search citation (w)

In [18]:
npl_abs = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher,
        TLS214_NPL_PUBLN.npl_page_first,
        TLS214_NPL_PUBLN.npl_page_last,
        TLS214_NPL_PUBLN.npl_abstract_nr
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_abstract_nr != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_abs)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date,npl_publn_end_date,npl_publisher,npl_page_first,npl_page_last,npl_abstract_nr
0,005E87824CE6FA317940AED91BEBE317,None,j,"PATENT ABSTRACTS OF JAPAN vol. 008, no. 178 (C...",None,None,PATENT ABSTRACTS OF JAPAN,008,None,178,19840816,None,None,None,None,(C - 238)
1,009C15C816E6700154E7B12BC8B446FD,None,d,DATABASE WPI Derwent World Patents Index; AN 9...,None,None,None,None,None,None,None,None,DATABASE WPI,None,None,93-148739
2,018F1999AFCF5126B704E8C8424E1D36,None,j,"PATENT ABSTRACTS OF JAPAN vol. 009, no. 245 (M...",None,None,PATENT ABSTRACTS OF JAPAN,009,None,245,19851002,None,None,None,None,(M - 418)
3,0196E27569BA05D4270BB533C64FBB39,None,d,"DATABASE WPI Week 201438, Derwent World Patent...",None,None,None,2014,None,38,None,None,DATABASE WPI,None,None,2014-L04303
4,01D8C95930399C2D84766B17D8BFF676,None,d,"DATABASE WPI Week 201115, Derwent World Patent...",None,None,None,2011,None,15,None,None,DATABASE WPI,None,None,2011-B36265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,FE9660E3A29113BD0F37B66D80FF015C,None,j,"PATENT ABSTRACTS OF JAPAN vol. 013, no. 349 (P...",None,None,PATENT ABSTRACTS OF JAPAN,013,None,349,19890807.0,None,None,None,None,(P - 911)
736,FEF91DFB3DA27E922D2EE1728AC0688E,None,j,"PATENT ABSTRACTS OF JAPAN vol. 9, no. 51 (M - ...",None,None,PATENT ABSTRACTS OF JAPAN,9,None,51,19850306.0,None,None,None,None,(M - 361)<1774>
737,FF206D89AE27A919B8DBCA562D51D469,None,d,DATABASE WPI Derwent World Patents Index; AN 9...,None,None,None,None,None,None,None,None,DATABASE WPI,None,None,90-167868
738,FF48181FD1D6DD23C00CABB9CA7CEF94,None,j,"PATENT ABSTRACTS OF JAPAN vol. 011, no. 088 (E...",None,None,PATENT ABSTRACTS OF JAPAN,011,None,088,19870318.0,None,None,None,None,(E - 490)


### NPL_DOI: 
Digital Object Identifier for direct online access. This attribute may only be populated for these NPL types:
- Book citation (b)
- Serial / Journal / Periodical citation (s) 
- World Wide Web / Internet search citation (w)

In [20]:
npl_doi = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher,
        TLS214_NPL_PUBLN.npl_page_first,
        TLS214_NPL_PUBLN.npl_page_last,
        TLS214_NPL_PUBLN.npl_abstract_nr,
        TLS214_NPL_PUBLN.npl_doi
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_doi != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_doi)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date,npl_publn_end_date,npl_publisher,npl_page_first,npl_page_last,npl_abstract_nr,npl_doi
0,00218FFFF69C484923E71C185D811395,None,s,"WILLIAMS , JOHN: 'Quantitative method for the ...","WILLIAMS , JOHN",Quantitative method for the profiling of the e...,Analytical Chemistry,79,None,15,20070000,None,None,5582,5593,None,doi:10.1021/AC0624086
1,002783E3EB8C9B96A3F5D94DA63E6327,None,s,BANDAL V ET AL: 'Output Feedback Fuzzy Sliding...,BANDAL V ET AL,Output Feedback Fuzzy Sliding Mode Control Tec...,"INDUSTRIAL TECHNOLOGY, 2005. ICIT 2005. IEEE I...",None,None,None,20051214,None,None,341,346,None,10.1109/ICIT.2005.1600661
2,0050BD5D59AD49B09E206AE065D05EAD,None,w,TEMBURNIKAR. KARTIK W. ET AL.: 'Antiproliferat...,TEMBURNIKAR. KARTIK W. et al.,Antiproliferative activities of halogenated th...,Bioorganic & Medicinal Chemistry.,22,None,None,20140303,None,None,2113,2122,None,doi:10.1016/j.bmc.2014.02.033
3,00582ED3DC3AAA88631B31012158693B,None,s,NA WANG: 'Lidar-Assisted Wind Turbine Feedforw...,Na Wang,Lidar-Assisted Wind Turbine Feedforward Torque...,2014 American Control Conference(ACC),None,None,None,20140604,None,None,3728,3733,None,10.1109/ACC.2014.6859039
4,006F5D23266B815F7227AD08A67E5B1D,None,s,HOWEY D A ET AL: 'Design and performance of a ...,HOWEY D A ET AL,Design and performance of a centimetre-scale s...,"SMART MATERIALS AND STRUCTURES, IOP PUBLISHING...",20,None,8,20110720,None,None,85021,None,None,10.1088/0964-1726/20/8/085021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3409,FF383005BBDF52458ABE4F244181EED9,None,s,LIU YONG ET AL: 'Design Considerations for MHz...,LIU YONG ET AL,Design Considerations for MHz PCB Winding Magn...,PROCEEDINGS OF THE 2019 IEEE 4TH INTERNATIONAL...,None,None,None,20191125.0,None,None,1,8,None,10.1109/IFEEC47410.2019.9015122
3410,FF384BCAF3197F48946896D333ECD887,None,s,R.C. SMITH ET AL: 'Improved Method of Setting ...,R.C. SMITH ET AL,Improved Method of Setting Successful Cement P...,JOURNAL OF PETROLEUM TECHNOLOGY,36,None,11,19841101.0,None,None,1897,1904,None,10.2118/11415-PA
3411,FF62D5F4BE20513AFB093FC923ECEBC0,None,s,"YANG TING, ZHAO LIYUAN; LIU YACHUANG; FENG SHA...","Yang Ting, Zhao Liyuan; Liu Yachuang; Feng Sha...",Dynamic Economic Dispatch for Integrated Energ...,"Automation of Electric Power Systems, NANJING ...",45,None,5,20201112,None,None,39,47,None,10.7500/AEPS20200405004
3412,FF91C236B1A923D15682F0DDCAB02411,None,s,SAID SYED A M ET AL: 'The effect of environmen...,SAID SYED A M ET AL,The effect of environmental factors and dust a...,RENEWABLE AND SUSTAINABLE ENERGY REVIEWS,82,None,None,20180228,None,None,743,760,None,10.1016/J.RSER.2017.09.042


### NPL_ISBN and NPL_ISSN: 
Identifiers for books and journals. This attribute may only be populated for these NPL types:
- Book citation (b)
- Chemical Abstract citation (c) (if ISSN)
- Biological abstract citation (i) (if ISSN)
- Patent Abstracts of Japan (j) 
- Serial / Journal / Periodical citation (s)
- Derwent citation (d) (if ISSN)
- World Wide Web / Internet search citation (w)

In [21]:
npl_issbn = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher,
        TLS214_NPL_PUBLN.npl_page_first,
        TLS214_NPL_PUBLN.npl_page_last,
        TLS214_NPL_PUBLN.npl_abstract_nr,
        TLS214_NPL_PUBLN.npl_doi,
        TLS214_NPL_PUBLN.npl_isbn,
        TLS214_NPL_PUBLN.npl_issn
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.npl_issn != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_issbn)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date,npl_publn_end_date,npl_publisher,npl_page_first,npl_page_last,npl_abstract_nr,npl_doi,npl_isbn,npl_issn
0,00582ED3DC3AAA88631B31012158693B,None,s,NA WANG: 'Lidar-Assisted Wind Turbine Feedforw...,Na Wang,Lidar-Assisted Wind Turbine Feedforward Torque...,2014 American Control Conference(ACC),None,None,None,20140604,None,None,3728,3733,None,10.1109/ACC.2014.6859039,None,0004245104
1,006CB6DDD3170568AFE2D22533DAFB8E,None,s,"'COMBINED PROPULSION PLANT CHANGES GEAR.', MOT...",None,COMBINED PROPULSION PLANT CHANGES GEAR.,"MOTORSHIP., NEXUS MEDIA COMMUNICATIONS, SWANLE...",75.,None,885.,19940401,None,None,12/13.,None,None,None,None,0027-2000
2,006F5D23266B815F7227AD08A67E5B1D,None,s,HOWEY D A ET AL: 'Design and performance of a ...,HOWEY D A ET AL,Design and performance of a centimetre-scale s...,"SMART MATERIALS AND STRUCTURES, IOP PUBLISHING...",20,None,8,20110720,None,None,85021,None,None,10.1088/0964-1726/20/8/085021,None,0964-1726
3,007056A3CE2F95F7268BD49D44D02E42,None,s,DEZHEN SONG ET AL: 'A Low False Negative Filte...,DEZHEN SONG ET AL,A Low False Negative Filter for Detecting Rare...,"IEEE TRANSACTIONS ON IMAGE PROCESSING, IEEE SE...",19,None,9,20100901,None,None,2321,2331,None,10.1109/TIP.2010.2048151,None,1057-7149
4,0091BE1239545B3401BE5BC7166537BC,None,s,H. SPÄTH ET AL.: 'Short-time backup- power for...,H. SPÄTH et al.,Short-time backup- power for mains-fed dc-link...,35th Annual IEEE Power Electronics Specialist ...,None,None,None,20040620,None,None,2475,2481,None,None,0-7803-8399-0,0275-9306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644,FF62D5F4BE20513AFB093FC923ECEBC0,None,s,"YANG TING, ZHAO LIYUAN; LIU YACHUANG; FENG SHA...","Yang Ting, Zhao Liyuan; Liu Yachuang; Feng Sha...",Dynamic Economic Dispatch for Integrated Energ...,"Automation of Electric Power Systems, NANJING ...",45,None,5,20201112,None,None,39,47,None,10.7500/AEPS20200405004,None,1000-1026
2645,FF685E102CA37CEB5A0AB46E342D0A5E,None,s,HARDESTY R M ET AL: 'LIDAR MEASUREMENT OF TURB...,HARDESTY R M ET AL,LIDAR MEASUREMENT OF TURBULENCE ENCOUNTERED BY...,"JOURNAL OF ATMOSPHERIC AND OCEANIC TECHNOLOGY,...",4,None,None,19870301,None,None,191,203,None,None,None,0739-0572
2646,FF6FC571E4DA3E315741787C6D233E04,None,s,PHADKE A G ET AL: 'A New Measurement Technique...,PHADKE A G ET AL,A New Measurement Technique for Tracking Volta...,IEEE TRANSACTIONS ON POWER APPARATUS AND SYSTE...,PAS-100,None,5,19830501.0,None,None,1025,1038,None,None,None,0018-9510
2647,FF91C236B1A923D15682F0DDCAB02411,None,s,SAID SYED A M ET AL: 'The effect of environmen...,SAID SYED A M ET AL,The effect of environmental factors and dust a...,RENEWABLE AND SUSTAINABLE ENERGY REVIEWS,82,None,None,20180228,None,None,743,760,None,10.1016/J.RSER.2017.09.042,None,1364-0321


### ONLINE_AVAILABILITY: 
URL or link for accessing the document online. This attribute may only be populated for these NPL types:
- Database citation (e) 
- World Wide Web / Internet search citation (w)

In [22]:
npl_online = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher,
        TLS214_NPL_PUBLN.npl_page_first,
        TLS214_NPL_PUBLN.npl_page_last,
        TLS214_NPL_PUBLN.npl_abstract_nr,
        TLS214_NPL_PUBLN.npl_doi,
        TLS214_NPL_PUBLN.npl_isbn,
        TLS214_NPL_PUBLN.npl_issn,
        TLS214_NPL_PUBLN.online_availability
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.online_availability != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_online)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,npl_publn_date,npl_publn_end_date,npl_publisher,npl_page_first,npl_page_last,npl_abstract_nr,npl_doi,npl_isbn,npl_issn,online_availability
0,0003BCB82D667DABC8B4C0C7CE188095,None,w,BORUP M ; ET AL: 'General rights Copyright and...,Borup M ; ET AL,General rights Copyright and moral rights for ...,None,None,None,None,20020101,None,None,1,7,None,None,None,None,https://core.ac.uk/download/pdf/13780517.pdf
1,0068AD434D52BE0F0A85FDC49CCA9317,None,w,'MWPS Currently Available Second Hand Wind Tur...,None,MWPS Currently Available Second Hand Wind Turb...,None,None,None,None,20121101,None,None,1,4,None,None,None,None,http://www.docstoc.com/docs/145755817/MWPS-Sec...
2,006FA797DBC97EE50821C26629701AC7,None,w,GANTASALA SUDHAKAR ET AL: 'Detection of blade ...,Gantasala Sudhakar ET AL,Detection of blade icing and its influence on ...,None,None,None,None,20190101,None,None,None,None,None,None,978-91-7-790483-0,None,http://www.diva-portal.org/smash/get/diva2:136...
3,00B91BF54A66F3464DEE97616A1BB616,None,w,ANONYMOUS: 'Laser cladding technology helps ma...,Anonymous,Laser cladding technology helps manufacturers ...,laserfocusworld,None,None,None,20180301,None,None,None,None,None,None,None,None,https://www.laserfocusworld.com/industrial- la...
4,00D0195BEA61B610F76BB18A6BB2B6C1,None,w,RYCROFT MIKE: 'Enabling wind turbines to opera...,Rycroft Mike,Enabling wind turbines to operate at high wind...,None,None,None,None,20150410,None,None,None,None,None,None,None,None,https://www.ee.co.za/article/enabling-wind-tur...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,FEA92DDFD9E336CC43EB802EB7788FFC,None,w,M FRIEDMAN ET AL: 'Low-loss RF transport over ...,M Friedman ET AL,Low-loss RF transport over long distances,IEEE Transactions on Microwave Theory and Tech...,None,None,None,20010101.0,None,None,341,348,None,10.1109/22.903095,None,None,http://ieeexplore.ieee.org/ielx5/22/19537/0090...
1830,FEB03E72D60A44CD376C1934EF7C9487,None,w,CHRISTIAN BARTSCH: 'Offshore-Windpark RIFFGAT'...,Christian Bartsch,Offshore-Windpark RIFFGAT,None,None,None,None,20130820,None,None,None,None,None,None,None,None,https://youtu.be/GWMeSRouTPw
1831,FED40AE4BC3BA512EB4D205567804513,None,w,ALVING & FREEBERG: 'The effect of riblets on s...,Alving & Freeberg,The effect of riblets on sails,None,None,None,None,19950615.0,None,None,None,None,None,None,None,None,http://rd.springer.com/article/10.1007%2FBF001...
1832,FFC4A7139DFB31B2BFB27D158F81C1A1,None,w,NADJA SKOPLJAK: 'TWD and Barge Master Float Co...,Nadja Skopljak,TWD and Barge Master Float Cost Saving Monopil...,Offshore Wind,None,None,None,20160810.0,None,None,1,2,None,None,None,None,https://www.offshorewind.biz/2016/08/10/twd-an...


### ONLINE_CLASSIFICATION: 
This attribute represents an online classification code, which provides thematic or categorical information about the Non-Patent Literature. This attribute may only be populated for these NPL types:
- Derwent citation (d)

In [23]:
npl_class = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume, 
        TLS214_NPL_PUBLN.npl_editor, 
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher,
        TLS214_NPL_PUBLN.npl_page_first,
        TLS214_NPL_PUBLN.npl_page_last,
        TLS214_NPL_PUBLN.npl_abstract_nr,
        TLS214_NPL_PUBLN.npl_doi,
        TLS214_NPL_PUBLN.npl_isbn,
        TLS214_NPL_PUBLN.npl_issn,
        TLS214_NPL_PUBLN.online_availability,
        TLS214_NPL_PUBLN.online_classification
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.online_classification != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_class)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,...,npl_publn_end_date,npl_publisher,npl_page_first,npl_page_last,npl_abstract_nr,npl_doi,npl_isbn,npl_issn,online_availability,online_classification
0,039E761F496A9AAE0CEE636C95B66197,None,d,"DATABASE WPI Week 200365, Derwent World Patent...",None,None,None,2003,None,65,...,None,DATABASE WPI,None,None,2003-681879,None,None,None,None,A32
1,05A0FABCF432EA010B154D2403474544,None,d,"DATABASE WPI Week 197739, Derwent World Patent...",None,None,None,1977,None,39,...,None,DATABASE WPI,None,None,1977-69306Y,None,None,None,None,A32
2,091B849378D4219D90730D622B89CC33,None,d,"DATABASE WPI Week 198417, Derwent World Patent...",None,None,None,1984,None,17,...,None,DATABASE WPI,None,None,1984-105287,None,None,None,None,X12
3,09BCE7207FB3D6BE0BDC77744B146B57,None,d,"DATABASE WPI Section Ch Week 199827, Derwent W...",None,None,None,1998,None,27,...,None,DATABASE WPI,None,None,1998-300852,None,None,None,None,A17
4,0D1C75C66713B189724789FC6D17DB8C,None,d,"DATABASE WPI Week 197702, Derwent World Patent...",None,None,None,1977,None,02,...,None,DATABASE WPI,None,None,1977-A3762A,None,None,None,None,P32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,F31CFB91438D18467220249A7F10233D,None,d,DATABASE WPI Derwent World Patents Index; Clas...,None,None,None,None,None,None,...,None,DATABASE WPI,None,None,1997-498794/46,None,None,None,None,X15
115,F7CF1E1821686FEF2617F68318BADC17,None,d,"DATABASE WPI Week 200620, Derwent World Patent...",None,None,None,2006,None,20,...,None,DATABASE WPI,None,None,2006-187409,None,None,None,None,A85
116,F858A66D1A27D4DD4D90B96239B34D89,None,d,"DATABASE WPI Section EI Week 201653, Derwent W...",None,None,None,2016,None,53,...,None,DATABASE WPI,None,None,2016-478741,None,None,None,None,U12
117,FA9408F45DD5AA6622F09953CA1AD564,None,d,"DATABASE WPI Section PQ Week 9027, Derwent Wor...",None,None,None,90,None,27,...,None,DATABASE WPI,None,None,90-208457,None,None,None,None,P13


### ONLINE_SEARCH_DATE: 
Date on which the document was retrieved online. This attribute may only be populated for these NPL types:
- World Wide Web / Internet search citation (w)

In [24]:
npl_records = (
    db.query(
        TLS214_NPL_PUBLN.npl_publn_id,
        TLS214_NPL_PUBLN.xp_nr,
        TLS214_NPL_PUBLN.npl_type,
        TLS214_NPL_PUBLN.npl_biblio, 
        TLS214_NPL_PUBLN.npl_author,
        TLS214_NPL_PUBLN.npl_title1,
        TLS214_NPL_PUBLN.npl_title2,
        TLS214_NPL_PUBLN.npl_volume,
        TLS214_NPL_PUBLN.npl_editor,
        TLS214_NPL_PUBLN.npl_issue,
        TLS214_NPL_PUBLN.npl_publn_date,
        TLS214_NPL_PUBLN.npl_publn_end_date,
        TLS214_NPL_PUBLN.npl_publisher,
        TLS214_NPL_PUBLN.npl_page_first,
        TLS214_NPL_PUBLN.npl_page_last,
        TLS214_NPL_PUBLN.npl_abstract_nr,
        TLS214_NPL_PUBLN.npl_doi,
        TLS214_NPL_PUBLN.npl_isbn,
        TLS214_NPL_PUBLN.npl_issn,
        TLS214_NPL_PUBLN.online_availability,
        TLS214_NPL_PUBLN.online_classification,
        TLS214_NPL_PUBLN.online_search_date
    )
    .filter(TLS214_NPL_PUBLN.npl_publn_id != '0')
    .filter(TLS214_NPL_PUBLN.online_search_date != None)
    .order_by(TLS214_NPL_PUBLN.npl_publn_id)
)

npl_res = patstat.df(npl_records)
npl_res

,npl_publn_id,xp_nr,npl_type,npl_biblio,npl_author,npl_title1,npl_title2,npl_volume,npl_editor,npl_issue,...,npl_publisher,npl_page_first,npl_page_last,npl_abstract_nr,npl_doi,npl_isbn,npl_issn,online_availability,online_classification,online_search_date
0,0003BCB82D667DABC8B4C0C7CE188095,None,w,BORUP M ; ET AL: 'General rights Copyright and...,Borup M ; ET AL,General rights Copyright and moral rights for ...,None,None,None,None,...,None,1,7,None,None,None,None,https://core.ac.uk/download/pdf/13780517.pdf,None,20210621
1,0068AD434D52BE0F0A85FDC49CCA9317,None,w,'MWPS Currently Available Second Hand Wind Tur...,None,MWPS Currently Available Second Hand Wind Turb...,None,None,None,None,...,None,1,4,None,None,None,None,http://www.docstoc.com/docs/145755817/MWPS-Sec...,None,20140121
2,006FA797DBC97EE50821C26629701AC7,None,w,GANTASALA SUDHAKAR ET AL: 'Detection of blade ...,Gantasala Sudhakar ET AL,Detection of blade icing and its influence on ...,None,None,None,None,...,None,None,None,None,None,978-91-7-790483-0,None,http://www.diva-portal.org/smash/get/diva2:136...,None,20210226
3,00B91BF54A66F3464DEE97616A1BB616,None,w,ANONYMOUS: 'Laser cladding technology helps ma...,Anonymous,Laser cladding technology helps manufacturers ...,laserfocusworld,None,None,None,...,None,None,None,None,None,None,None,https://www.laserfocusworld.com/industrial- la...,None,20221031
4,00CE5A87311FDDC866AE0BFBDD7C2EF9,None,w,GILL A F ET AL: 'Effect of variation in fibre ...,GILL A F ET AL,Effect of variation in fibre volume fraction o...,"COMPOSITES SCIENCE AND TECHNOLOGY, ELSEVIER, UK",69,None,14,...,None,2368,2375,None,10.1016/J.COMPSCITECH.2009.02.008,None,0266-3538,None,None,20090214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,FEA92DDFD9E336CC43EB802EB7788FFC,None,w,M FRIEDMAN ET AL: 'Low-loss RF transport over ...,M Friedman ET AL,Low-loss RF transport over long distances,IEEE Transactions on Microwave Theory and Tech...,None,None,None,...,None,341,348,None,10.1109/22.903095,None,None,http://ieeexplore.ieee.org/ielx5/22/19537/0090...,None,20151218
1733,FEB8FF27666BF8F51906D668B594F493,None,w,WECKX SAM ET AL: 'Load Balancing With EV Charg...,WECKX SAM ET AL,Load Balancing With EV Chargers and PV Inverte...,"IEEE TRANSACTIONS ON SUSTAINABLE ENERGY, IEEE,...",6,None,2,...,None,635,643,None,10.1109/TSTE.2015.2402834,None,1949-3029,None,None,20150318
1734,FED40AE4BC3BA512EB4D205567804513,None,w,ALVING & FREEBERG: 'The effect of riblets on s...,Alving & Freeberg,The effect of riblets on sails,None,None,None,None,...,None,None,None,None,None,None,None,http://rd.springer.com/article/10.1007%2FBF001...,None,20130703
1735,FFC4A7139DFB31B2BFB27D158F81C1A1,None,w,NADJA SKOPLJAK: 'TWD and Barge Master Float Co...,Nadja Skopljak,TWD and Barge Master Float Cost Saving Monopil...,Offshore Wind,None,None,None,...,None,1,2,None,None,None,None,https://www.offshorewind.biz/2016/08/10/twd-an...,None,20230420
